# FARGOpy
## Wrapping FRAGO3D

## Tutorial: flux computation

<a target="_blank" href="https://colab.research.google.com/github/seap-udea/fargopy/blob/main/dev/fargopy-flux_test_refactor.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### What's in this notebook

In this notebook we illustrate **configuration variables** in `FARGOpy`.

### Before starting

If you are in `Google Colab`, install the latest version of the package:

In [1]:
# @title Install fargopy
import sys
if 'google.colab' in sys.modules:
    !sudo pip install -Uq fargopy

For this tutorial you will need the following external modules and tools:

In [1]:
import fargopy as fp
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

%load_ext autoreload
%autoreload 2

Running FARGOpy version 0.3.15


### Cargar datos

In [2]:
fp.Simulation.download_precomputed('p3disoj')
sim = fp.Simulation(output_dir='/tmp/p3disoj')

Precomputed output directory '/tmp/p3disoj' already exist
FARGO3D directory '/Users/jzuluaga/fargo3d/' does not exist.
Now you are connected with output directory '/tmp/p3disoj'
Found a variables.par file in '/tmp/p3disoj', loading properties
Loading variables
85 variables loaded
Simulation in 3 dimensions
Loading domain in spherical coordinates:
	Variable phi: 128 [[0, np.float64(-3.117048960983623)], [-1, np.float64(3.117048960983623)]]
	Variable r: 64 [[0, np.float64(0.5078125)], [-1, np.float64(1.4921875)]]
	Variable theta: 32 [[0, np.float64(1.4231400767948967)], [-1, np.float64(1.5684525767948965)]]
Number of snapshots in output directory: 11
Configuration variables and domains load into the object. See e.g. <sim>.vars


## Interpolación en XYZ

La interpolación ahora se puede realizar en 3 dimensiones espaciales

Creamos la instancia de la clase DataHandler y cargamos los datos de la simulación:

Así estaba: 

```python
data_handler = DataHandler(sim) # Instanciamos el manejador de datos
data_handler.load_data(
   # Definimos el snapshot inicial y el final sobre el que queremos trabajar
    snapi=1,
    snapf=10
);
```

Así me lo sueño:

In [ ]:
# Carga normal 
# gasdens = sim.load_field('gasdens', snapshot=1, interpolate=False)
# Carga con interpolación
gasdens_interp = sim.load_field('gasdens', snapshot=(1, 10), interpolate=True) # Ahora gasdens es un objeto DataHandler
velocity_interp = sim.load_field('gasv', snapshot=(1, 10), interpolate=True)

# Cargar varios campos
gasens_interp, velocity_interp = sim.load_field(['gasdens','gasv'], snapshot=(1, 10), interpolate=True)

# Cargar los datos
gasens, velocity = sim.load_field(['gasdens','gasv'], snapshot=1, interpolate=False)

Veamos de que tipos son:

In [ ]:
type(gasens), type(gasdens_interp)

Sugiero: DataHandler -> FieldInterpolator

Original: 

```python
# Interpolamos el campo de densidad"

# Definimos las coordenadas y el tiempo en el que queremos interpolar
time=0.54
x=1.0
y=-0.25
z=0.05

# Interpolamos el campo de densidad
density = data_handler.interpolate_field(time,x,y,z,field_name="gasdens")
print(f"La densidad en las coordenadas {(x,y,z)} en el tiempo:{time}\n es : {density*sim.URHO*1e3:.3e}[kg/m^3]")

# Interpolar el campo de velocidad
velx,vely,velz = data_handler.interpolate_velocity(time,x,y,z)
```

Que me sueño:

Definimos las coordenadas y el tiempo en el que queremos interpolar

In [4]:
time=0.54 # This is the time of the claculation of the flux (it is not required to be a time of a specific snapshot)

# Coordinates where I will calculate the field
x=1.0
y=-0.25
z=0.05

Interpolamos el campo de densidad"

In [ ]:
# Interpolamos el campo de densidad
density = gasden_interp.evaluate(time=time, x=x, y=y, z=z)
print(f"La densidad en las coordenadas {(x,y,z)} en el tiempo:{time}\n es : {density*sim.URHO*1e3:.3e}[kg/m^3]")

# Interpolamos el campo de densidad
density = gasden_interp.evaluate(snapshot=5, x=x, y=y, z=z)
print(f"La densidad en las coordenadas {(x,y,z)} en el tiempo:{time}\n es : {density*sim.URHO*1e3:.3e}[kg/m^3]")

# Interpolar el campo de velocidad
velx,vely,velz = velocity_interp.evaluate(time, x=x, y=y, z=z)
```

Este era el orginal:

```python
from fargopy.flux import FluxAnalyzer3D
from fargopy import flux
import fargopy as fp
from fargopy.Fsimulation import DataHandler
%load_ext autoreload
%autoreload 2


# Cambiar dir_path por la ruta donde se tienen los outputs de Fargo3D
output_dir = '/tmp/p3disoj' # Cambiar por la ruta donde se tienen los outputs de Fargo3D
sim = fp.Simulation()


# Creamos el objeto
analyzer = FluxAnalyzer3D(
    output_dir=output_dir, # Cambiar por la ruta donde se tienen los outputs de Fargo3D
    sphere_center=(1, 0, 0), # Centro de la esfera (centrada en el planeta)
    radius=0.08,    # Radio de la esfera
    subdivisions=6,     # Número de subdivisiones
    snapi=1,        # Snapshot inicial
    snapf=10       # Snapshot final
)
```

Este es el sueño (caso de uso):

Crear la superficie de observación:

In [ ]:
sphere = fp.Sphere(
    sphere_center=(1, 0, 0), # Centro de la esfera (centrada en el planeta)
    radius=0.08,    # Radio de la esfera
    subdivisions=6,     # Número de subdivisiones
)

Mostrar qué es lo que creaste:

In [ ]:
type(sphere), sphere.center, sphere.radius, sphere.subdivisions

Gráfico de la esfera:

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(6,6))

cmap = 'viridis'
ax = axs

# Heat map of the vertical component of velocity
c = ax.pcolormesh(mesh.x*sim.UL/fp.AU,mesh.z*sim.UL/fp.AU,gasvz_planet*sim.UV/(1e5),cmap=cmap)
cbar = fig.colorbar(c)
cbar.set_label('$v_z$ [km/s]')

# Idea de como meter una superficie de control dentro de un gráfico
sphere.plot(ax=ax)

ax.set_xlabel('$x$ [au]')
ax.set_ylabel('$z$ [au]')
fp.Plot.fargopy_mark(ax)

Calculamos flujo:

In [ ]:
flux_analyzer = fp.FluxAnalyzer3D(
    simulation=sim,
    surface=sphere,
    #function=density*velocity*area
    desnity=gasdens_interp,
    velocity=velocity_interp,
)

analyzer.calculate_fluxes() # Escondes debajo la interpolación

Existe una alternativa:

In [ ]:
# Flujo 
vector_function = fp.FieldIntegrals(
    simulation=sim,
    surface=sphere,
    type='surface',
    integrand=lambda f1,f2: f1*f2, # \int f1*f2*dS 
    fields=[gasdens_interp, velocity_interp],
)
vector_function.evaluate(time=0.4)

# Masa
vector_function = fp.FieldIntegrals(
    simulation=sim,
    surface=sphere,
    tyep='volume',
    integrand=lambda f1: f1, # \int f1*dV
    fields=[gasdens_interp],
)
vector_function.evaluate(time=0.4)
